# Triangular Arbitrage — BTC/ETH/USDT (toy)
We compute synthetic cross rates, detect opportunities, and simulate execution with fees/latency.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
np.random.seed(3)
T=3000
btc_usdt=68000+np.cumsum(np.random.randn(T)*5)
eth_usdt=3700+np.cumsum(np.random.randn(T)*0.5)
btc_eth=btc_usdt/eth_usdt*(1+0.0005*np.random.randn(T))
fee=0.0004


## 1. Cycle profit factor

In [ ]:
cycle = btc_usdt/(eth_usdt*btc_eth)
edge = cycle*(1-fee)**3 - 1


## 2. Signals when edge>threshold

In [ ]:
thr=0.0008
sig=(edge>thr).astype(int)


## 3. Execution simulator with latency

In [ ]:
lat=3
pnl=np.zeros(T)
pos=0.0
for t in range(T):
    if t>=lat and sig[t-lat]==1:
        pnl[t]+=edge[t]*100.0 # notional
pnl=np.cumsum(pnl)


## 4. Plots & stats

In [ ]:
plt.figure(); plt.plot(edge); plt.title('Edge');
plt.figure(); plt.plot(pnl); plt.title('PnL'); plt.show()
print('final pnl',pnl[-1])


## 5. Sensitivity to fee/latency

In [ ]:
for f in [0.0002,0.0004,0.0006,0.001]:
    e=cycle*(1-f)**3-1
    print(f,' final edge mean', float(np.mean(e)))


## 6. Robustness via bands

In [ ]:
import numpy as np
mu=np.convolve(edge,np.ones(200)/200,'same')
sig=np.sqrt(np.convolve((edge-mu)**2,np.ones(200)/200,'same')+1e-12)
z=(edge-mu)/(sig+1e-9)
print('z>2 fraction', np.mean(z>2))


## 7. Position scaling by edge

In [ ]:
scale = np.clip((edge-thr)/thr,0,1)
pnl2=np.cumsum(scale*edge*100.0)
plt.figure(); plt.plot(pnl2); plt.title('Scaled PnL'); plt.show()


## 8. Drawdown

In [ ]:
dd=np.maximum.accumulate(pnl2)-pnl2
print('max DD', np.max(dd))


## 9. Takeaways
## 10. Next steps